<a href="https://colab.research.google.com/drive/1dC6KP-6rDM_Hy6oUOqPWT2l9ywIz2qmK?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Setup***

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.strings import regex_replace
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.optimizers import Adam

# ***Preparing the Data***

In [2]:
def prepareData(dir):
  data = text_dataset_from_directory(dir)
  return data.map(
    lambda text, label: (regex_replace(text, '<br />', ' '), label),
)

train_data = prepareData(r"C:\Users\feras\Desktop\All\Research Notebooks\movie-reviews-dataset\train")
test_data = prepareData(r"C:\Users\feras\Desktop\All\Research Notebooks\movie-reviews-dataset\test")

for text_batch, label_batch in train_data.take(1):
  print(text_batch.numpy()[0])
  print(label_batch.numpy()[0])

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
b'Actually, they don\'t, but they certainly did when trying to think of a singular line that adequately summarises how terrible this entry in the series really is. There were some moments that could have been good, but they are mostly outweighed by their own conversion into missed opportunities, and don\'t get me started on the bad.  The wasted opportunities are pretty obvious, but I will recap them here in case anyone cares. Anyone who hasn\'t seen the film and genuinely gives a toss would be advised to stop reading at this point. The first, and potentially the biggest, wasted opportunity, was the plot with Freddy\'s long-lost child. Now, the extreme mental illness that Freddy appears to suffer (and I might hasten to add that less than one percent of mental patients are a threat to other people, leave alone to this extent) is HEREDITARY, so why not a mystery-type slasher in which Lisa Zane\'s character

# ***Building the Model***

In [3]:
model = Sequential()
model.add(Input(shape=(1,), dtype="string"))

# Text Vectorization
max_tokens = 1000
max_len = 100
vectorize_layer = TextVectorization(
  max_tokens=max_tokens,
  output_mode="int",
  output_sequence_length=max_len,
)

train_texts = train_data.map(lambda text, label: text)
vectorize_layer.adapt(train_texts)

model.add(vectorize_layer)

# Embedding
model.add(Embedding(max_tokens + 1, 128))

# Recurrent Layer (Long Short-Term Memory (LSTM))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64, dropout=0.25, recurrent_dropout=0.25))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

# ***Compiling the Model***

In [4]:
model.compile(
  optimizer=Adam(learning_rate=0.005),
  loss='binary_crossentropy',
  metrics=['accuracy'],
)

# ***Training the Model***

In [5]:
model.fit(train_data, epochs=10)

Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 73s 80ms/step - accuracy: 0.5667 - loss: 0.6757
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 59s 75ms/step - accuracy: 0.7349 - loss: 0.5456
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 59s 76ms/step - accuracy: 0.7766 - loss: 0.4807
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 58s 73ms/step - accuracy: 0.7981 - loss: 0.4472
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 67s 85ms/step - accuracy: 0.8074 - loss: 0.4257
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 51s 66ms/step - accuracy: 0.8175 - loss: 0.4122
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 51s 65ms/step - accuracy: 0.8225 - loss: 0.4003
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 51s 65ms/step - accuracy: 0.8262 - loss: 0.3906
Epoch 9/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 51s 65ms/step - accuracy: 0.8256 - loss: 0.3845
Epoch 10/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 51s 65ms/step - accuracy: 0.8371 - loss: 0.3682


# ***Using the Model***

In [6]:
model.save_weights('rnn.weights.h5')
model.load_weights('rnn.weights.h5')

# ***Testing the Model***

In [7]:
model.evaluate(test_data)

782/782 ━━━━━━━━━━━━━━━━━━━━ 46s 57ms/step - accuracy: 0.8010 - loss: 0.4225


[0.4301472306251526, 0.7976400256156921]